# Portfolio Workflow

In [1]:
import logging
import logging.config

logging.config.fileConfig('./config/logging.ini')
logger = logging.getLogger('GatherPriceHistories')

import configparser
from platform import python_version
from pathlib import Path

# Set the import path for the project tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, 'tools')

# Project imports
import importlib
import utils
importlib.reload(utils)

from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import pandas_datareader as pdr

from tqdm.notebook import tqdm

logger.info(f'Python version: {python_version()}')
logger.info(f'Pandas version: {pd.__version__}')
logger.info(f'Pandas Data Reader version: {pdr.__version__}')

2022-04-23 17:35:19,345|numexpr.utils|INFO|NumExpr defaulting to 4 threads.
2022-04-23 17:35:20,371|GatherPriceHistories|INFO|Python version: 3.8.8
2022-04-23 17:35:20,372|GatherPriceHistories|INFO|Pandas version: 1.3.5
2022-04-23 17:35:20,373|GatherPriceHistories|INFO|Pandas Data Reader version: 0.10.0


In [2]:
config = configparser.ConfigParser()
config.read('./config/config.ini')
default_config = config["DEFAULT"]

# Stage 1: Generate Stock Universe

- Gather stocks from specific criteria (SP500 top 50...)
- Use stock sentiment to select stocks
- Gather price histories

## Stock Universe

Here we setup the univers. This needs some work. The long term goal is to use a pipeline process to help select stock that are in the top 500 or something similare.

For now we will use stocks from the portfolio, but stocks of interest (high news items), a list of well known stocks (this also has been augmented with some stocks that made Ameritrade's top 10 movers for a couple of days. This Ameritrade funciton has not been coded yet, but should be add down the line to automate pulling these tickers.

# Price History data

One you have a set of investments you want to work with, you will need to pull some historical data for them.

We will obtain 5 years of price histories. In the end this will provide us with 2 years of factor data since some of the factors are based on 1 year returns.

In [ ]:
# Make sure we have a data directory
Path(default_config["DataDirectory"]).mkdir(parents=True, exist_ok=True) 
price_histories_file_name = default_config["DataDirectory"] + '/' + default_config["PriceHistoriesFileName"]
logger.info(f'Storing {default_config["NumberOfYearsPriceHistories"]} years of price histories in: {price_histories_file_name}')

logger.info('Gathering S&P 500 stock tickers...')
snp_500_stocks = utils.get_snp500()
logger.info(f'Gathered {len(snp_500_stocks)} S&P 500 stock tickers.')
logger.info('Reducing S&P 500 stock tickers based on finvis sentiment...')
stock_universe = utils.reduce_universe_by_sentiment(set(snp_500_stocks.index.to_list()))
logger.info(f'Reduced to {len(stock_universe)} S&P 500 stock tickers.')

start = datetime.today() - relativedelta(years = int(default_config["NumberOfYearsPriceHistories"]))
end = datetime.today() - relativedelta(days = 1)
logger.info(f'Reading price histories from {start} to {end} using Yahoo Daily Reader...')
yahoo = pdr.yahoo.daily.YahooDailyReader(symbols=stock_universe, start=start, end=end, adjust_price=True, interval='d', get_actions=False, adjust_dividends=True)
price_histories = yahoo.read()
yahoo.close()
logger.info(f'Read {len(price_histories)} price histories.')
logger.info(f'PRICE_HISTORIES_FILE|{price_histories_file_name}')
price_histories.to_csv(price_histories_file_name, index=True)
logger.info('Price histories saved.')
logger.info(f'PRICE_HISTORIES|{price_histories.index.min()}|{price_histories.index.max()}')

2022-04-23 17:35:20,428|GatherPriceHistories|INFO|Storing 5 years of price histories in: ./data/price_histories_yahoo.csv
2022-04-23 17:35:20,431|GatherPriceHistories|INFO|Gathering S&P 500 stock tickers...
2022-04-23 17:35:21,189|GatherPriceHistories|INFO|Gathered 504 S&P 500 stock tickers.
2022-04-23 17:35:21,189|GatherPriceHistories|INFO|Reducing S&P 500 stock tickers based on finvis sentiment...


Tickers:   0%|          | 0/504 [00:00<?, ?Finvis Postings/s]

News Tables:   0%|          | 0/502 [00:00<?, ?News Table Items/s]

2022-04-23 17:37:36,527|GatherPriceHistories|INFO|Reduced to 431 S&P 500 stock tickers.
2022-04-23 17:37:36,527|GatherPriceHistories|INFO|Reading price histories from 2017-04-23 17:37:36.527394 to 2022-04-22 17:37:36.527394 using Yahoo Daily Reader...
